In [1]:
from cadcad.spaces import space
from cadcad.dynamics import block
from cadcad.spaces import Bit, Real, Integer
from cadcad.points import Point

import numpy as np

In this notebook i will make a discrete time variant of a homocidal chaufeur

In [2]:
@space
class CartesianPlane:
    x:float
    y:float

CartesianPlaneDot = CartesianPlane.rename_dims({'x':'dx', 'y':'dy'})

@space
class Particle:
    pos:CartesianPlane
    vel:CartesianPlaneDot

print(Particle)
Particle.unroll_schema()

Space Particle has dimensions {'pos': 'CartesianPlane', 'vel': 'CartesianPlane'}


{'pos': {'x': 'float', 'y': 'float'}, 'vel': {'dx': 'float', 'dy': 'float'}}

In [3]:
CartesianPlaneDot.unroll_schema()

{'dx': 'float', 'dy': 'float'}

In [4]:
@space
class PolarPlane:
    r:float
    theta:float

PolarPlaneDot = PolarPlane.rename_dims({'r':'dr', 'theta':'dtheta'})

@space
class Unicycle:
    pos: PolarPlane
    vel: PolarPlaneDot

print(Unicycle)
Unicycle.unroll_schema()

Space Unicycle has dimensions {'pos': 'PolarPlane', 'vel': 'PolarPlane'}


{'pos': {'r': 'float', 'theta': 'float'},
 'vel': {'dr': 'float', 'dtheta': 'float'}}

In [5]:
PolarPlaneDot.unroll_schema()

{'dr': 'float', 'dtheta': 'float'}

In [6]:
@space
class GameState:
    alice: Particle
    bob: Unicycle

@space
class NormalizedGameState:
    alice: Particle
    bob: Particle

print(GameState)
GameState.unroll_schema()

Space GameState has dimensions {'alice': 'Particle', 'bob': 'Unicycle'}


{'alice': {'pos': {'x': 'float', 'y': 'float'},
  'vel': {'dx': 'float', 'dy': 'float'}},
 'bob': {'pos': {'r': 'float', 'theta': 'float'},
  'vel': {'dr': 'float', 'dtheta': 'float'}}}

In [7]:
initAlice = Point(Particle, {'pos': {'x': 1.0, 'y': 1.0},
 'vel': {'dx': -1.0, 'dy': 0.0}})

initBob = Point(Unicycle, {'pos': {'r': 0.1, 'theta': np.pi/4},
 'vel': {'dr': 1.0, 'dtheta': 0.0}})

init = Point(GameState, {'alice':initAlice.data, 'bob':initBob.data})

In [8]:
print(init)

Point in space GameState has data
{
    "alice": {
        "pos": {
            "x": 1.0,
            "y": 1.0
        },
        "vel": {
            "dx": -1.0,
            "dy": 0.0
        }
    },
    "bob": {
        "pos": {
            "r": 0.1,
            "theta": 0.7853981633974483
        },
        "vel": {
            "dr": 1.0,
            "dtheta": 0.0
        }
    }
}



Alice is a Particle Trying to Evade the Unicycle Bob so she will attempt to maximize her distance from Bob in the Next timesteo

In [9]:
PolarPlaneDot.unroll_schema()

{'dr': 'float', 'dtheta': 'float'}

In [10]:
PolarPlane.unroll_schema()

{'r': 'float', 'theta': 'float'}

In [11]:
@block
def awkwardConverter1(domain:Point[PolarPlaneDot])->Point[PolarPlane]:
    """"""
    input = domain
    output = {}
    output['r'] = input['dr']
    output['theta'] = input['dtheta']

    return Point(PolarPlane, output)
    
@block
def awkwardConverter2(domain:Point[CartesianPlane])->Point[CartesianPlaneDot]:
    """needs """
    input = domain
    output = {}
    output['dx'] = input['x']
    output['dy'] = input['y']

    return Point(CartesianPlaneDot, output)

In [12]:
@block
def polar2cartesian(domain: Point[PolarPlane]) -> Point[CartesianPlane]:
    
    r = domain["r"]
    theta = domain["theta"]

    x = r*np.cos(theta)
    y = r*np.sin(theta)

    output = {"x":x, "y":y}

    return Point(CartesianPlane, output)

@block
def cartesian2polar(domain: Point[CartesianPlane]) -> Point[PolarPlane]:
    
    x = domain["x"]
    y = domain["y"]

    r = np.sqrt(x**2+y**2)
    if x==0:
        theta = np.sign(y)*np.pi/2
    else:
        theta = np.arctan(y/x)

    output = {"r":r, "theta":theta}

    return Point(PolarPlane, output)

@block
def normalize(domain:Point[GameState])->Point[NormalizedGameState]:
    input=domain
    aliceData = input['alice']

    bobPosPolarPt = Point(PolarPlane, input['bob']['pos'])
    bobVelPolarDotPt = Point(PolarPlaneDot, input['bob']['vel'])
    bobVelPolarPt = awkwardConverter1(bobVelPolarDotPt)

    bobPosCartesian = polar2cartesian(bobPosPolarPt)
    bobVelCartesianTemp = polar2cartesian(bobVelPolarPt)
    print(bobVelCartesianTemp)
    bobVelCartesian=awkwardConverter2(bobVelCartesianTemp)

    bobData = {'pos': bobPosCartesian, 'vel': bobVelCartesian}

    output = Point(NormalizedGameState, {'alice':aliceData, 'bob':bobData})

    return output



In [13]:
normalizedInit = normalize(init)

Point in space CartesianPlane has data
{
    "x": 1.0,
    "y": 0.0
}



ValueError: Schema mismatch between the Point's Space and the data given. Expected {'dr': <class 'float'>, 'dtheta': <class 'float'>}, but received ["dx -> <class 'numpy.float64'>", "dy -> <class 'numpy.float64'>"]